# Week 9-1: Unsupervised Machine Learning

## Bibliotheken importeren

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.simplefilter('ignore')
import sqlite3
go_sales_connect = sqlite3.connect('go_sales.sqlite')
go_crm_connect = sqlite3.connect('go_crm.sqlite')

## Data inlezen en kolommen selecteren

In [ ]:
df = pd.read_sql_query("SELECT sales_branch.SALES_BRANCH_CODE, CITY, REGION, sales_branch.COUNTRY_CODE, COUNT(ORDER_NUMBER) from sales_branch join country on country.COUNTRY_CODE = sales_branch.COUNTRY_CODE join order_header on order_header.SALES_BRANCH_CODE = sales_branch.SALES_BRANCH_CODE GROUP BY sales_branch.SALES_BRANCH_CODE", go_sales_connect)
df2 = pd.read_sql_query("SELECT country_code, sales_territory_code from country", go_crm_connect)
df = pd.merge(df, df2, on='COUNTRY_CODE', how='left')
df

In [ ]:
# df['ADDRESS1'] = df['ADDRESS1'].astype(str)
# df.dtypes

## One-hot encoding van onafhankelijke niet-numerieke variabelen

In [ ]:
# df['Pclass'] = df['Pclass'].astype(str)
# df.dtypes

In [ ]:
dummies_df1 = pd.get_dummies(df.loc[:, ["SALES_BRANCH_CODE",	"CITY",	"REGION",	"COUNTRY_CODE",	"COUNT(ORDER_NUMBER)",	"SALES_TERRITORY_CODE"]])
dummies_df1

In [ ]:
# df = pd.concat([df, dummies_dataframe], axis = 1)
# df = df.drop(['Sex'], axis = 1)
# df = df.loc[:, ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Age', 'Survived']]
# df

## Clusteringmodel bouwen met 2 dimensies

In [ ]:
# df_2d = df.loc[:, ['Age', 'Survived']]
# df_2d

In [ ]:
kmeans = KMeans(n_clusters = 5, random_state = 42)
kmeans.fit_predict(dummies_df1)

In [ ]:
kmeans_centra = pd.DataFrame(kmeans.cluster_centers_)
kmeans_centra

In [ ]:
for i in range(len(kmeans_centra.columns)):
    kmeans_centra = kmeans_centra.rename(columns = {i : f'{dummies_df1.columns[i]}'})
    
kmeans_centra

In [ ]:
for src_index, _ in dummies_df1.iterrows():
    euclidian_distances = dict()
    print(f"Afstand van bronindex {src_index} tot...")

    for centrumindex, _ in kmeans_centra.iterrows():
        print(f"\tCentrumindex {centrumindex}:")
        euclidian_sum = 0

        for column_name in kmeans_centra.columns:
            current_difference = dummies_df1.at[src_index, column_name] - kmeans_centra.at[centrumindex, column_name]
            print(f'\t\t{dummies_df1.at[src_index, column_name]} - {kmeans_centra.at[centrumindex, column_name]} = {current_difference}')
            euclidian_sum += current_difference ** 2
        
        print(f'\tTotale euclidische som: {euclidian_sum}')
        euclidian_distance = math.sqrt(euclidian_sum)
        print(f'\tEuclidische afstand: {euclidian_distance}')
        euclidian_distances[centrumindex] = euclidian_distance
        print('------------------------------------------------')
    
    print(euclidian_distances)
    centrum_number = min(euclidian_distances, key = euclidian_distances.get)
    print(centrum_number)
    dummies_df1.at[src_index, 'Centrum'] = centrum_number
    print("================================================")

dummies_df1

In [ ]:
# Interclusterafstand
for i in kmeans_centra.index:
    for j in kmeans_centra.index:
        if i != j:
            diff = kmeans_centra.loc[i] - kmeans_centra.loc[j]
            dist = np.sqrt(np.sum(diff**2))
            print(f"Interclusterafstand tussen cluster {i} en cluster {j}: {dist}")

# Intraclusterafstand
for i in kmeans_centra.index:
    cluster_points = dummies_df1[dummies_df1['Centrum'] == i]
    distances = []
    for _, point in cluster_points.iterrows():
        diff = point[:-1] - kmeans_centra.loc[i]
        dist = np.sqrt(np.sum(diff**2))
        distances.append(dist)
    avg_distance = np.mean(distances)
    print(f"Gemiddelde intraclusterafstand voor cluster {i}: {avg_distance}")



In [ ]:
plt.scatter(dummies_df1['SALES_TERRITORY_CODE_1'], dummies_df1['Centrum'], color = 'k')
plt.show()

In [ ]:
dummies_df1.groupby('Centrum', as_index = False)['Centrum'].count()

## Clusteringmodel bouwen met meer dan 2 dimensies (alle kolommen uit de dataset)

### Centra berekenen

In [ ]:
kmeans = KMeans(n_clusters = 6, random_state = 42)
prediction_results = kmeans.fit_predict(dummies_df1)
prediction_results

In [ ]:
df['Centrum'] = prediction_results
df

### De juiste centra toewijzen aan rijen uit de dataset

In [ ]:
df.groupby('Centrum', as_index = False)['Centrum'].count()

## Evalueren